### Import modules

In [1]:
import mmap
import shutil
import os
import re
import sqlite3
import subprocess
from sqlite3 import Error
from pathlib import Path

### Global constants

In [21]:
NETTIME_FOLDER = r'C:\Program Files\SPEC\netTimeV4'
APP_NAME = 'netTimeV4'
SERVICE = 'netTimeV4'
APP = 'netTime'
V55_RELEASE = 'netTime 5.5.1.16168'
INSTALLER = 'netTime_x64_es_es.msi'

EMPLOYEES = 540
ACCESS_EMPLOYEES = 540

### Stop service

In [4]:
def change_service(service='netTimeV4', command='start'):
    try:
        output = subprocess.check_output(["net", command, service])
    except Exception as e:
        output = str(e)
    return output

In [149]:
change_service(service='netTime', command='stop')

"Command '['net', 'stop', 'netTime']' returned non-zero exit status 2."

### Last log

In [6]:
def get_last_log(app_folder=NETTIME_FOLDER):
    # set logs folder
    dirpath = os.path.join(app_folder, 'Logs')
    # get all files order by date
    paths = sorted(Path(dirpath).iterdir(), key=os.path.getmtime)
    # safety only
    if not paths:
        return None
    # return last
    return paths[-1]

### License with last log

In [7]:
def get_license(logfile=get_last_log()):
    # empty default
    nro_lic = ''
    # open file
    with open(logfile, 'rb', 0) as file, \
        mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ) as s:
        # search licencia
        pos = s.find(b'Licencia: ')
        # if founded, set posic and read number
        if pos:
            s.seek(pos + 10)
            nro_lic = s.read(5).decode()
    # return lic
    return nro_lic

In [8]:
LICENCIA = get_license()
LICENCIA

'BQS0W'

### Copy structure v4_to_v5 to nettime folder

In [9]:
def copy_structure(app_folder=NETTIME_FOLDER):
    orig = os.path.join(os.path.dirname(os.getcwd()), 'v4_to_v5_5')
    dest = os.path.join(nt_folder, 'v4_to_v5_5')

    try:
        shutil.rmtree(dest)
    except FileNotFoundError:
        pass

    # copy data
    return shutil.copytree(orig, dest)

In [42]:
copy_structure()

'C:\\Program Files\\SPEC\\netTimeV4\\v4_to_v5_5'

### Make necessary backups

In [48]:
def make_backup(app_folder, app_name='netTimeV4'):
    # dest bkp folder
    seg_folder = os.path.join(app_folder, 'v4_to_v5_5', '1_SegDatosV4toV5', app_name)
    
    try:
        # remove old bkps
        shutil.rmtree(seg_folder)
        
        # move bkp folder
        shutil.move(
            os.path.join(nt_folder, 'backup'),
            os.path.join(seg_folder, 'backup')
        )
    except FileNotFoundError:
        pass

    try:
        # folders to bkp
        exclude = [
            'backup',
            'v4_to_v5_5'
        ]
        
        # files to bkp
        files = [file for file in os.listdir(nt_folder) if file not in exclude]
        
        for file in files:
            # copy path
            if os.path.isdir(os.path.join(nt_folder, file)):
                # copy path
                shutil.copytree(
                    os.path.join(nt_folder, file),
                    os.path.join(seg_folder, file)
                )
            # copy file
            elif os.path.isfile(os.path.join(nt_folder, file)):
                shutil.copy2(
                    os.path.join(nt_folder, file),
                    seg_folder
                )
        
        # general propose
        return True
    
    except Exception as error:
        return error

In [49]:
make_backup(app_folder=NETTIME_FOLDER, app_name=APP_NAME)

True

### Remove database files for active license

In [10]:
def remove_needs(app_folder, app='netTime'):
    # files to removes
    files = [
        f'{app}.mov',
        f'{app}.sl3',
        f'{app}.sl3-journal',
        'workflow.sl3'
    ]
    for file in files:
        if os.path.exists(os.path.join(app_folder, file)):
            os.remove(os.path.join(app_folder, file))
            
    # folders to remove
    folders = [
        'backup'
    ]
    for folder in folders:
        if os.path.isdir(os.path.join(app_folder, folder)):
            _files = Path(os.path.join(app_folder, folder)).iterdir()
            for f in _files:
                os.remove(f)
                
    return True

In [55]:
remove_needs(app_folder=NETTIME_FOLDER, app=APP)

### Run installer

In [58]:
def run_installer(version='netTime 5.5.1.16168', installer='netTime_x64_es_es.msi'):
    print("1- Completar la instalación.")
    print("2- Activar la licencia.")
    print("3- Detener el servicio.")
    
    file = os.path.join(os.path.dirname(os.getcwd()), version, installer)
    os.startfile(file)
    
    input('4- Presiona Enter al finalizar...')

In [59]:
run_installer(version=V55_RELEASE, installer=INSTALLER)
# wait user with input() ...

1- Completa la instalación.
2- Activa la licencia.
3- Deten el servicio.
4- Presiona Enter al finalizar...


### Stop service

In [125]:
change_service(service=SERVICE, command='stop')

"b''"

#### Remove inst backups and database

In [66]:
# remove files again
remove_needs(app_folder=NETTIME_FOLDER, app=APP)

#### Get last backup (not-used)

In [12]:
def get_last_bkp(app_folder, app_name='netTimeV4'):
    bkp_fodler = os.path.join(
        app_folder,
        'v4_to_v5_5',
        '1_SegDatosV4toV5',
        app_name,
        'backup'
    )
    # list backup elements
    items = sorted(Path(bkp_fodler).iterdir(), key=os.path.getmtime, reverse=True)
    
    # get last config
    for item in items:
        if re.match('config_backup_\d*_\d*\.zip', item.name):
            element = item
            break
    else:
        element = None
    
    # return found element or None
    return element

### Restore databases from backups

In [132]:
def db_recovery(app_folder, app='netTime'):
    # dest bkp folder
    seg_folder = os.path.join(app_folder, 'v4_to_v5_5', '1_SegDatosV4toV5', f'{app}V4')
    
    sl3 = [
        f'{app}.mov',
        f'{app}.sl3',
        'workflow.sl3'
    ]
    
    # restore databases
    for file in sl3:
        # ensure no files
        if os.path.exists(os.path.join(app_folder, file)):
            os.remove(os.path.join(app_folder, file))
        
        # restore
        shutil.copy2(
            os.path.join(seg_folder, file),
            nt_folder
        )
        
    # general propose
    return True

In [133]:
db_recovery(app_folder=NETTIME_FOLDER, app=APP)

True

## Scripts

In [29]:
def verify_access(app_folder, app='netTime', access_number=None):
    db_file = os.path.join(app_folder, f'{app}.sl3')
    
    with sqlite3.connect(db_file) as conn:
        cursor = conn.cursor()
        
        # lic accesos
        cursor.execute("SELECT Count(*) FROM APPDATA WHERE IDTYPE = 14 AND DATA LIKE '%parent=\"PerfilAccesos\"%';")
        accesos = cursor.fetchone()
        accesos = accesos[0] if accesos else -1
        
    # if match with saved data
    if accesos == access_number:
        return True
    
    # in difference case
    return False

In [32]:
if not verify_access(app_folder=NETTIME_FOLDER, app=APP, access_number=ACCESS_EMPLOYEES):
    decic = input('El número de empleados con accesos no coincide.\r\n\
    Desea continuar? (Y/N)')
    
    if decic.lower() == 'y':
        print('Continuando...')

El número de empleados con accesos no coincide.
    Desea continuar? (Y/N)n


In [136]:
def query_auto_generate(errors, app_folder, app='netTime'):
    # database file
    db_file = os.path.join(app_folder, f'{app}.sl3')
    
    # out queries
    queries = []
    
    with sqlite3.connect(db_file) as conn:
        cursor = conn.cursor()
        
        for error in errors:
            # build and execute select query
            cursor.execute(
                "{}{}{}".format(
                    "SELECT DATA FROM APPDATA WHERE DATA LIKE '%",
                    error,
                    "%' AND IDTYPE != 27 LIMIT 1;"
                )
            )
            # view one result only
            result = cursor.fetchone()
            if not result:
                print("Error getting data with", error)
                continue
            
            # find datetime in text
            re_match = re.search(f'E{error[1:]}[\d\:\.]*', result[0])
            if not re_match:
                re_match = re.search(f'e{error[1:]}[\d\:\.]*', result[0])
                if not re_match:
                    print("Error getting hour.", error, result[0], sep="\n")
                    continue
            
            # str match
            match = result[0][re_match.start(): re_match.end()]
            
            # split to fix hours to 00:00...
            m_split = match.split('T')
            hour = re.sub('\d', '0', m_split[1])
            
            # upper or lower case
            tr = 'endDate="2040-12-31T'
            if match.find("EndDate") >= 0:
                tr = 'EndDate="2040-12-31T'
            
            # query build
            new_query = '{}{}{}{}{}{}'.format(
                "UPDATE APPDATA SET DATA = REPLACE(DATA,'",
                match,
                "','",
                tr,
                hour,
                "') WHERE IDTYPE != 27;"
            )
            
            # append to generated queries
            #print("Adding", new_query)
            queries.append(new_query)
        
    # return builds
    return queries

In [145]:
def run_scripts(app_folder, app='netTime', queries=[]):
    db_file = os.path.join(app_folder, f'{app}.sl3')
    
    with sqlite3.connect(db_file) as conn:
        cursor = conn.cursor()

        # run update last date commands
        for command in queries:
            cursor.execute(command)
        
        # commit changes
        if queries:
            conn.commit()
        
        # verify results
        ld_results = {'_ok': [], '_error': [], '_exec_queries': queries}
        
        # list of verifies
        ld_verify_commands = [
            'EndDate="2019-12-30T',
            'EndDate="2019-12-31T',
            'EndDate="2020-01-01T',
            'EndDate="2020-12-31T',
            'EndDate="2030-12-31T',
            'endDate="2019-12-30T',
            'endDate="2019-12-31T',
            'endDate="2020-01-01T',
            'endDate="2020-12-31T',
            'endDate="2030-12-31T',
        ]
        
        # build and execute verifies
        for command in ld_verify_commands:
            cursor.execute(
                "{}{}{}".format(
                    "SELECT Count(*) FROM APPDATA WHERE DATA LIKE '%",
                    command,
                    "%' AND IDTYPE != 27;"
                )
            )
            
            result = cursor.fetchone()
            if result:
                if result[0] == 0:
                    ld_results['_ok'].append(command)
                else:
                    ld_results['_error'].append(command)
    
    if ld_results.get('_error'):
        print("Error en fechas. Recursivando...")
        
        # recursive call if errors
        return run_scripts(
            app_folder=app_folder,
            app=app,
            queries=query_auto_generate(
                errors=ld_results.get('_error'),
                app_folder=app_folder,
                app=app
            )
        )
    
    # return resume
    return ld_results

In [146]:
run_scripts(app_folder=NETTIME_FOLDER, app=APP)

Error en fechas. Recursivando...
Error en fechas. Recursivando...


{'_ok': ['EndDate="2019-12-30T',
  'EndDate="2019-12-31T',
  'EndDate="2020-01-01T',
  'EndDate="2020-12-31T',
  'EndDate="2030-12-31T',
  'endDate="2019-12-30T',
  'endDate="2019-12-31T',
  'endDate="2020-01-01T',
  'endDate="2020-12-31T',
  'endDate="2030-12-31T'],
 '_error': [],
 '_exec_queries': ['UPDATE APPDATA SET DATA = REPLACE(DATA,\'endDate="2030-12-31T00:00:00.000\',\'endDate="2040-12-31T00:00:00.000\') WHERE IDTYPE != 27;',
  'UPDATE APPDATA SET DATA = REPLACE(DATA,\'endDate="2030-12-31T00:00:00.000\',\'endDate="2040-12-31T00:00:00.000\') WHERE IDTYPE != 27;']}

### LastYear to .ini

In [147]:
def update_ini(app_folder, app='netTime'):
    
    # function to match setting
    def match_setting(element):
        if re.match('Configuration\.LastYear\=\d*', element):
            return element
    
    # setting
    setting = 'Configuration.LastYear='
    
    # set path
    ini_path = os.path.join(app_folder, f'{app}.ini')
    
    # set vars
    value = None
    
    # open file
    with open(ini_path, 'r') as f:
        content = f.readlines()
    
    # remove if founded
    found = list(filter(match_setting, content))
    if found:
        content.remove(found[0])
    
    # add setting
    content.append(f'{setting}2040\n')
    
    # open file
    with open(ini_path, 'w') as f:
       f.writelines(content)
        
    return True

In [148]:
update_ini(app_folder=NETTIME_FOLDER, app=APP)

True

### Unificar diferencias en .ini

In [108]:
input('Unifique .ini y .exe.config con los backups...\n\
Presione Enter para continuar...')

Unifique .ini y .exe.config con los backups...
Presione Enter para continuar...


''

### Start service

In [20]:
change_service(service='netTime', command='start')

b'El servicio de netTime est\xa0 deteni\x82ndose.\r\nEl servicio de netTime se detuvo correctamente.\r\n\r\n'